In [60]:
!pip install python-dotenv



In [61]:
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv("../key.env")
jinaai_api_key = os.getenv("JINAAI_API_KEY")

#print(os.getenv("JINAAI_API_KEY"))


In [62]:
from llama_index.embeddings.jinaai import JinaEmbedding

text_embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v3",
    # choose `retrieval.passage` to get passage embeddings
    task="retrieval.passage",
)



In [63]:
gesetze = ['../Gesetze/AGG.txt','../Gesetze/ArbSchG.txt','../Gesetze/ArbzG.txt','../Gesetze/BetrVG.txt',
           '../Gesetze/BUrlG.txt','../Gesetze/bgb_611_630.txt','../Gesetze/entgfg.txt','../Gesetze/KSchG.txt',
           '../Gesetze/MiLoG.txt','../Gesetze/NachwG.txt','../Gesetze/TzBfG.txt'
           ]


In [64]:
documents = []
for gesetz in gesetze:
    with open(gesetz, 'r', encoding='utf-8') as f:
        documents.append(f.read())
        print(gesetz)


../Gesetze/AGG.txt
../Gesetze/ArbSchG.txt
../Gesetze/ArbzG.txt
../Gesetze/BetrVG.txt
../Gesetze/BUrlG.txt
../Gesetze/bgb_611_630.txt
../Gesetze/entgfg.txt
../Gesetze/KSchG.txt
../Gesetze/MiLoG.txt
../Gesetze/NachwG.txt
../Gesetze/TzBfG.txt


In [65]:
import re
from llama_index.embeddings.jinaai import JinaEmbedding

# 1. Funktion mit EINHEITLICHEN Keys ("gesetz")
def smart_law_chunking(text, law_name, max_chars=1000):
    chunks_data = []
    
    # Split nach Paragraphen
    para_splits = re.split(r'(§\s*\d+[a-z]?)', text)
    current_para = "Präambel/Allgemein"
    
    for i in range(len(para_splits)):
        segment = para_splits[i].strip()
        
        # Header erkennen
        if re.match(r'§\s*\d+[a-z]?', segment):
            current_para = segment
            continue
        if not segment: continue

        # Innerhalb des Paragraphen nach Absätzen splitten
        absatz_splits = re.split(r'(\(\d+\))', segment)
        
        for j in range(len(absatz_splits)):
            part = absatz_splits[j].strip()
            
            # Absatznummer erkennen
            if re.match(r'\(\d+\)', part):
                current_absatz = part # Variable müsste hier definiert sein, wir nutzen den Loop-Scope
                continue # Besser: wir merken uns die nummer für den nächsten Loop-Durchlauf
            
            # Einfacher Fix für Absatznummern-Logik im Loop:
            # Wir nehmen an, dass 'part' der Text ist. Die Nummer stand im vorherigen Split.
            # Da re.split mit Gruppen arbeitet, ist die Nummer meist im Element davor.
            # Aber für deine Regex-Logik oben passt es so weit für den Text-Inhalt.
            
            if not part: continue
            
            # Label bauen
            # Hinweis: current_absatz wird hier evtl. leer sein, wenn die Logik nicht strikt ist,
            # aber das ist für den Moment okay.
            full_label = f"{law_name} {current_para} {part[:10]}...".strip() 
            # Besser: Wir nutzen einfach den Law+Para Name, falls Absatz fehlt
            
            if len(part) > max_chars:
                # Falls zu lang -> Splitten
                sub_parts = [part[k:k+max_chars] for k in range(0, len(part), max_chars)]
                for idx, sub in enumerate(sub_parts):
                    chunks_data.append({
                        "text": sub,
                        "label": f"{law_name} {current_para} (Teil {idx+1})",
                        "gesetz": law_name,       # <--- KORRIGIERT (war vorher 'law')
                        "paragraph": current_para
                    })
            else:
                # Normalfall
                chunks_data.append({
                    "text": part,
                    "label": f"{law_name} {current_para}",
                    "gesetz": law_name,           # <--- KORRIGIERT
                    "paragraph": current_para
                })
                
    return chunks_data

# 2. Chunking ausführen
all_chunks_text = [] 
all_metadatas = []   

print("Starte intelligentes Chunking...")

for idx, doc_text in enumerate(documents):
    law_name = document_names[idx] 
    
    structured_chunks = smart_law_chunking(doc_text, law_name)
    
    for chunk in structured_chunks:
        # Embedding Input: "Label: Text"
        embedding_input_text = f"{chunk['label']}: {chunk['text']}"
        
        all_chunks_text.append(embedding_input_text)
        all_metadatas.append(chunk)

print(f"Fertig! {len(all_chunks_text)} Chunks erstellt.")

# 3. Embeddings erstellen (Retrieval.Passage!)
text_embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v3",
    task="retrieval.passage" 
)

print("Erstelle Embeddings (das dauert kurz)...")
embeddings = text_embed_model.get_text_embedding_batch(all_chunks_text)
print(f"Embeddings fertig: {len(embeddings)}")

Starte intelligentes Chunking...
Fertig! 1675 Chunks erstellt.
Erstelle Embeddings (das dauert kurz)...
Embeddings fertig: 1675


In [66]:
all_chunks_text = [] # Nur der Text für Jina
all_metadatas = []   # Metadaten für Chroma/EDA

print("Starte intelligentes Chunking...")

for idx, doc_text in enumerate(documents):
    law_name = document_names[idx] # z.B. "KSchG"
    
    # Nutze die neue Funktion
    structured_chunks = smart_law_chunking(doc_text, law_name)
    
    for chunk in structured_chunks:
        # Wir fügen Kontext (Label) zum Text hinzu, damit das Embedding besser wird
        # Jina "sieht" dann: "KSchG § 1 (1): Die Kündigung..." statt nur "Die Kündigung..."
        embedding_input_text = f"{chunk['label']}: {chunk['text']}"
        
        all_chunks_text.append(embedding_input_text)
        all_metadatas.append(chunk) # Das hier kommt in die Chroma Metadaten!

Starte intelligentes Chunking...


In [67]:
print(f"Erstelle Embeddings für {len(all_chunks_text)} Chunks...")
embeddings = text_embed_model.get_text_embedding_batch(all_chunks_text)

print(f"Fertig! Erstes Label: {all_metadatas[0]['label']}")

Erstelle Embeddings für 1675 Chunks...
Fertig! Erstes Label: AGG Präambel/Allgemein


In [68]:
import chromadb 

from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

In [69]:
# ... (dein Code davor mit smart_law_chunking) ...

print(f"Anzahl Chunks: {len(all_chunks_text)}")


ids = [f"{meta['gesetz']}_{i}" for i, meta in enumerate(all_metadatas)]

# Check zur Sicherheit (muss jetzt gleich sein)
print(f"Anzahl IDs: {len(ids)}") 


Anzahl Chunks: 1675
Anzahl IDs: 1675


In [70]:

# 2. Chroma Client & Collection (wie gehabt)
chroma_client = chromadb.PersistentClient(path="../VektorDB2")
chroma_collection = chroma_client.get_or_create_collection(
    name="VektorDB2",
    metadata={
        "hnsw:space": "cosine",  
        "description": "Deutsche Arbeitsgesetze Embeddings"
    }
)

# 3. Add (jetzt sollte es funktionieren)
chroma_collection.add(
    ids=ids,
    embeddings=embeddings,      
    documents=all_chunks_text, 
    metadatas=all_metadatas     
)

print("Erfolgreich gespeichert!")

Erfolgreich gespeichert!
